![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/working-with-file-streams.png)

# Working With File Streams


In [ ]:
import azureml.dataprep as dprep

In addition to loading and parsing tabular data (see [here](./data-ingestion.ipynb) for more details), Data Prep also supports a variety of operations on raw file streams. 

File streams are usually created by calling `Dataflow.get_files`.

In [ ]:
dflow = dprep.Dataflow.get_files(path='../data/*.csv')
dflow.head(5)

The result of this operation is a Dataflow with a single column named "Path". This column contains values of type `StreamInfo`, each of which represents a different file matched by the search pattern specified when calling `get_files`. The string representation of a `StreamInfo` follows this pattern:

StreamInfo(_Location_://_ResourceIdentifier_\[_Arguments_\])

Location is the type of storage where the stream is located (e.g. Azure Blob, Local, or ADLS); ResouceIdentifier is the name of the file within that storage, such as a file path; and Arguments is a list of arguments required to load and read the file.

On their own, `StreamInfo` objects are not particularly useful; however, you can use them as input to other functions.

## Retrieving File Names

In the example above, we matched a set of CSV files by using a search pattern and got back a column with several `StreamInfo` objects, each representing a different file. Now, we will extract the file path and name for each of these values into a new string column.

In [ ]:
dflow = dflow.add_column(expression=dprep.get_stream_name(dflow['Path']),
                         new_column_name='FilePath',
                         prior_column='Path')
dflow.head(5)

The `get_stream_name` function will return the full name of the file referenced by a `StreamInfo`. In the case of a local file, this will be an absolute path. From here, you can use the `derive_column_by_example` method to extract just the file name.

In [ ]:
import os

first_file_path = dflow.head(1)['FilePath'][0]
first_file_name = os.path.basename(first_file_path)
dflow = dflow.derive_column_by_example(new_column_name='FileName',
                                       source_columns=['FilePath'],
                                       example_data=(first_file_path, first_file_name))
dflow = dflow.drop_columns(['FilePath'])
dflow.head(5)

## Writing Streams

Whenever you have a column containing `StreamInfo` objects, it's possible to write these out to any of the locations Data Prep supports. You can do this by calling `Dataflow.write_streams`:

In [ ]:
dflow.write_streams(streams_column='Path', base_path=dprep.LocalFileOutput('./test_out/')).run_local()

The `base_path` parameter specifies the location the files will be written to. By default, the name of the file will be the resource identifier of the stream with any invalid characters replaced by `_`. In the case of streams referencing local files, this would be the full path of the original file. You can also specify the desired file names by referencing a column containing them:

In [ ]:
dflow.write_streams(streams_column='Path',
                    base_path=dprep.LocalFileOutput('./test_out/'),
                    file_names_column='FileName').run_local()

Using this functionality, you can transfer files from any source to any destination supported by Data Prep. In addition, since the streams are just values in the Dataflow, you can use all of the functionality available.

Here, for example, we will write out only the files that start with the prefix "crime-". The resulting file names will have the prefix stripped and will be written to a folder named "crime".

In [ ]:
prefix = 'crime-'
dflow = dflow.filter(dflow['FileName'].starts_with(prefix))
dflow = dflow.add_column(expression=dflow['FileName'].substring(len(prefix)),
                         new_column_name='CleanName',
                         prior_column='FileName')
dflow.write_streams(streams_column='Path',
                    base_path=dprep.LocalFileOutput('./test_out/crime/'),
                    file_names_column='CleanName').run_local()